# Pirat spil
Denne Jupyter Notebook indeholder koden til et mini spil i JavaScript. Spillet er designet til at fungere med en YOLO machine learning model som kan genkende terninger fra et videofeed, hvor spillere kaster en terning og bevæger sig rundt på spillebrættet.


## Opsætter variabler:

In [ ]:
let totalCells = 34; // Antal felter på brættet

let boardSizeX = 3;
let boardSizeY = 10;

let cellSize = 60; // Størrelsen på hvert felt
let players = []; // Array til at gemme spillere
let currentPlayer = 0; // Indeks for den spiller, der har tur
let diceRoll = 1; // Terningens seneste kast
let colors = ['red', 'blue', 'green', 'yellow']; // Spillernes farver
let boardState = []; // Array til at holde styr på, hvor spillerne er
let backgroundImage; // Billedet til baggrunden

let windowWidthCustom = 1266;
let windowHeightCustom = 570;

let boardStartX = 320; // Startposition for spillebrættet
let boardStartY = 100;

let diceRemoved = false; // Flag til at holde øje med, om terningen er fjernet
let lastDiceRoll = null; // Gemmer det seneste terningekast

## Opsætning af spil


### preload()
Funktionen preload() bruges til at indlæse baggrundsbilledet, før spillet starter. Funktionen setup() opretter canvas med en specifik størrelse og initialiserer spillere og spillebrættet. Der bruges også en intervalfunktion til at tjekke for nye terningekast for hvert sekund.

### drawBoard()
Funktionen drawBoard() tegner felterne på spillebrættet i et mønster, der danner en rute. Funktionen drawCell() tegner hvert felt på brættet og placerer spillerne på deres respektive positioner.

### drawPlayers()
Funktionen drawPlayers() nulstiller spillebrættet og placerer spillerne på deres respektive felter igen.

### drawDiceDisplay()
Funktionen drawDiceDisplay() viser det seneste terningekast på skærmen i spillerens farve.

In [ ]:
function preload() {
    // Indlæser baggrundsbilledet, før spillet starter
    backgroundImage = loadImage('IMG_4723.png');
}

function setup() {
    // Opretter canvas med specifik størrelse
    createCanvas(windowWidthCustom, windowHeightCustom);

    // Initialiserer spillere med startposition 0 og en farve fra arrayet
    for (let i = 0; i < 4; i++) {
        players.push({ position: 0, color: colors[i] });
    }

    // Initialiserer spillebrættet med tomme felter
    for (let i = 0; i < totalCells; i++) {
        boardState.push([]);
    }

    // Tjekker for nye terningekast hver sekund
    setInterval(fetchDiceRoll, 1000);
}

function draw() {
    // Tegner baggrundsbilledet
    image(backgroundImage, 0, 0, width, height);

    drawBoard(); // Tegner spillebrættet
    drawPlayers(); // Opdaterer og tegner spillernes positioner
    drawDiceDisplay(); // Viser det seneste terningekast
}

function drawBoard() {
    noFill();
    stroke(0);
    strokeWeight(2);

    // Tegner felterne i et mønster, der danner en rute på spillebrættet
    for (let i = 0; i < 10; i++) {
        let index = i;
        drawCell(index, boardStartX + i * cellSize, boardStartY);
    }

    for (let i = 10; i < 12; i++) {
        let dy = i - 9;
        drawCell(i, boardStartX + 9 * cellSize, boardStartY + dy * cellSize);
    }

    for (let i = 12; i < 22; i++) {
        let reversedX = 21 - i;
        drawCell(i, boardStartX + reversedX * cellSize, boardStartY + 3 * cellSize);
    }

    for (let i = 22; i < 24; i++) {
        let dy = i - 21;
        drawCell(i, boardStartX, boardStartY + (3 + dy) * cellSize);
    }

    for (let i = 24; i < 34; i++) {
        let dx = i - 24;
        drawCell(i, boardStartX + dx * cellSize, boardStartY + 6 * cellSize);
    }
}

function drawCell(index, x, y) {
    push();
    rect(x, y, cellSize, cellSize); // Tegner firkanten for feltet

    let playersOnCell = boardState[index]; // Finder spillere på dette felt
    if (playersOnCell.length > 0) {
        let numPlayers = playersOnCell.length;
        if (numPlayers === 1) {
            noStroke();
            fill(playersOnCell[0]); // Farven for spilleren
            ellipse(x + cellSize / 2, y + cellSize / 2, 20, 20); // Tegner spiller
        } else {
            let angleStep = TWO_PI / numPlayers; // Vinkelafstand for flere spillere
            for (let j = 0; j < numPlayers; j++) {
                let angle = j * angleStep;
                let radius = cellSize / 4;
                let xOffsetPlayer = cos(angle) * radius;
                let yOffsetPlayer = sin(angle) * radius;
                noStroke();
                fill(playersOnCell[j]);
                ellipse(
                    x + cellSize / 2 + xOffsetPlayer,
                    y + cellSize / 2 + yOffsetPlayer,
                    20,
                    20
                );
            }
        }
    }
    pop();
}

function drawPlayers() {
    // Nulstiller spillebrættet
    boardState = [];
    for (let i = 0; i < totalCells; i++) {
        boardState.push([]);
    }

    // Placerer spillere på deres respektive felter
    for (let i = 0; i < players.length; i++) {
        let pos = players[i].position;
        if (pos >= totalCells) {
            pos = totalCells - 1; // Sikrer, at spillere ikke går uden for banen
            players[i].position = pos;
        }
        boardState[pos].push(players[i].color);
    }
}

function drawDiceDisplay() {
    // Viser terningekastet på skærmen i spillerens farve
    fill(players[currentPlayer].color);
    textSize(60);
    textAlign(CENTER, CENTER);
    text(`${diceRoll}`, width - 655, height - 350);}


## Hent terningeslag

### fetchDiceRoll()
Funktionen fetchDiceRoll() forespørger serveren efter det seneste terningekast og opdaterer spillebrættet baseret på kastet. Hvis terningen er fjernet, markeres den, og et nyt kast registreres kun, hvis terningen var fjernet.

### movePiece()
Funktionen movePiece() flytter den nuværende spillers brik fremad baseret på terningekastet og skifter tur til næste spiller.

In [ ]:
async function fetchDiceRoll() {
    try {
        // Forespørger serveren efter det seneste terningekast
        const response = await fetch("http://127.0.0.1:5000/dice");

        if (!response.ok) {
            if (response.status === 404) {
                // Hvis ingen ny terning er registreret, markeres den som fjernet
                if (!diceRemoved) {
                    diceRemoved = true;
                    console.log("Terning fjernet, venter på nyt kast...");
                }
                return;
            }
            throw new Error(`Serverfejl: ${response.status}`);
        }

        const data = await response.json();

        if (data.dice_roll !== undefined) {
            // Kun registrer et nyt kast, hvis terningen var fjernet
            if (diceRemoved) { 
                diceRoll = data.dice_roll;
                movePiece(diceRoll); // Flytter brikken baseret på kastet
                lastDiceRoll = data.dice_roll; // Opdaterer sidste kast
                diceRemoved = false; // Nulstiller markering for fjernet terning
            }
        }
    } catch (error) {
        console.error("Netværksfejl:", error.message);
    }
}

function movePiece(roll) {
    // Flytter den nuværende spillers brik fremad
    players[currentPlayer].position += roll;
    if (players[currentPlayer].position >= totalCells) {
        players[currentPlayer].position = totalCells - 1; // Begrænser til sidste felt
    }

    // Skifter tur til næste spiller
    currentPlayer = (currentPlayer + 1) % players.length;
}